In [3]:
import pyspark 
from pyspark.sql import SparkSession 
import time
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.config('spark.driver.memory' , '8g').getOrCreate()

In [5]:
best_rated = spark.read.parquet(r"D:\VS-workspace\bigdata-2025-1\Spark\output\best_rated\best_rated.parquet")

In [6]:
best_rated.show(20)

+--------------------+--------------------+-------------+-----+--------------------+-------------+----------------+------------+------------------+
|         business_id|                name|         city|state|          categories|total_reviews|avg_review_stars|total_useful|business_avg_stars|
+--------------------+--------------------+-------------+-----+--------------------+-------------+----------------+------------+------------------+
|1RqfozJoosHAsKZhc...|Walls Jewelry Rep...|    Nashville|   TN|Watch Repair, Loc...|          114|             5.0|          97|               5.0|
|-siOxQQcGKtb-04dX...|ella & louie flowers|Santa Barbara|   CA|Flowers & Gifts, ...|          104|             5.0|          65|               5.0|
|4-P4Bzqd01YvKX9tp...|       Drink & Learn|  New Orleans|   LA|Beer, Wine & Spir...|           90|             5.0|          47|               5.0|
|dhLARBhUnJloLa8xZ...|Steves iPhone Repair|  Cherry Hill|   NJ|Professional Serv...|           78|             5